In [85]:
examples = [
{
"quesition": "Top Gun",
"answer": """
🛩️👨‍✈️🔥
"""
},

{
"quesition": "대부",
"answer": """
👨‍👨‍👦🔫🍝
""",
},
]

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough


llm = ChatOpenAI(temperature=0.4)
memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100,
    memory_key="history_a",
    return_messages=True
)


example_prompt_1 = ChatPromptTemplate.from_messages([
    ("human", "{quesition}"),
    ("ai", "{answer}")
])

example_prompt_2 = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt_1,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 영화 전문가입니다. 영화 제목을 말하면 형식에 맞춰서 답해주세요. Top Gun, 대부는 예시입니다."),
    MessagesPlaceholder(variable_name="history_a"),
    example_prompt_2,
    ("human", "{quesition}?"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history_a"]

chain = (
    RunnablePassthrough.assign(history_a=load_memory)
    | final_prompt 
    | llm
)

def invoke_chain(quesition):
    result = chain.invoke({"quesition": quesition})
    memory.save_context({"input": quesition}, {"output": result.content})
    print(result.content)

invoke_chain("어벤져스")

🦸‍♂️🦸‍♀️🦹‍♂️🌍💥


In [87]:
invoke_chain("토르")

⚡️🔨🌩️


In [88]:
memory.load_memory_variables({})["history_a"]

[HumanMessage(content='어벤져스'),
 AIMessage(content='🦸\u200d♂️🦸\u200d♀️🦹\u200d♂️🌍💥'),
 HumanMessage(content='토르'),
 AIMessage(content='⚡️🔨🌩️')]

In [89]:
invoke_chain("처음 물어본 영화가 뭐야")

처음 물어본 영화는 "어벤져스"였습니다.
